In [11]:
import re
from nltk.corpus import stopwords
from unidecode import unidecode
from nltk.stem.snowball import ItalianStemmer
from collections import Counter
from wordcloud import WordCloud
import nltk
from IPython.display import Image 
import matplotlib.pyplot as plt

%matplotlib inline

nltk.download("stopwords") # nltk needs us to manually download the set of stopwords

def get_file_tokens(filename):
    tokens = []
    for line in filename:
        tokens += re.split('\W+', line,  flags=re.UNICODE) # re library deals with the splitting
    return tokens

def generate_tag_cloud(freq, image_filename=None):
	wc = WordCloud(background_color="white").generate_from_frequencies(freq)
	if image_filename:
        # If you pass it a filename, then it saves the image on the provided path
		image = wc.to_image()
		image.save(image_filename)
	else:
        # then it just displays it here on the notebook
		plt.imshow(wc, interpolation='bilinear')
		plt.axis("off")
		plt.show()
        
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('italian'))

def filter_words(words):
    return [w for w in words if len(w)>=3 and w not in STOPWORDS]

def normalize_words(words):
    return [unidecode(w.lower()) for w in words]

def stem_words(words):
	s = ItalianStemmer()
	return [s.stem(w) for w in words]

def get_stem_mapping(words):
	s = ItalianStemmer() 
	mapping = {} # here is our mapping
	for w in words: 
		stemmed_w = s.stem(w)
		if stemmed_w not in mapping: 
			mapping[stemmed_w] = Counter()
		mapping[stemmed_w].update([w])

	return mapping

def destem_words(stems, stem_mapping):
	return [stem_mapping[s].most_common(1)[0][0] for s in stems] # ask the counter to get the most common occurrence for each root word

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\RConti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
import os
import pandas as pd

def leggi_file_csv_in_sottocartelle(directory):
    # Lista dei DataFrame letti dai file CSV
    elenco_dataframe = []

    # Naviga attraverso tutte le sottocartelle e file nella directory data
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".csv"):
                # Costruisci il percorso completo del file
                file_path = os.path.join(root, file)
                # Leggi il file CSV e aggiungi il DataFrame alla lista
                df = pd.read_csv(file_path)
                elenco_dataframe.append(df)

    # Concatena tutti i DataFrame in un unico DataFrame
    dataframe_completo = pd.concat(elenco_dataframe, ignore_index=True)
    
    return dataframe_completo

# Specifica la directory radice in cui cercare i file CSV
directory_radice = 'C:/Users/RConti/PycharmProjects/pythonProject/PROGETTONE/CityNews_SCraper'

# Chiama la funzione per leggere e concatenare i file CSV nelle sottocartelle
tutti_i_dati = leggi_file_csv_in_sottocartelle(directory_radice)

In [13]:
import altair as alt
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import HBox, VBox, Layout
import warnings
warnings.filterwarnings("ignore")

# Sample DataFrame with longer category names
df = tutti_i_dati

# Dropdown options (list of Territorio)
dropdown_options = sorted(df['Edizione'].unique().tolist())

# Function to update chart1 based on selected category
def update_chart1(Edizione):
    filtered_df = df[df['Edizione'] == Edizione]

    Testo = filtered_df['Testo'].to_list()
    tokens = get_file_tokens(Testo)
    filtered_t = filter_words(normalize_words(tokens))
    stemmed = stem_words(filtered_t)
    stem_mapping = get_stem_mapping(filtered_t)
    destemmed = destem_words(stemmed, stem_mapping)
    c = Counter(destemmed)

    generate_tag_cloud(c)

    '''y = [count for tag, count in c.most_common(20)]
    x = [tag for tag, count in c.most_common(20)]
    
    plt.barh(x, y)
    plt.gca().invert_yaxis()
    plt.show()'''

    c_df = pd.DataFrame(list(c.items()), columns=['Word', 'Count'])

    # Sort the DataFrame by the 'Count' column in descending order
    df_sorted = c_df.sort_values(by='Count', ascending=False)

    # Select the top 20 rows
    top_20 = df_sorted.head(20)

    chart = alt.Chart(top_20).mark_bar().encode(
        x = 'Count:Q',
        y = alt.Y('Word:N', title='Word', sort='-x'),
        size=alt.Size(field='Count', type='quantitative', bin=alt.BinParams(maxbins=5)),
        color=alt.Color('Count:Q', scale=alt.Scale(scheme='blues')),
        tooltip=['Word', 'Count']
    )
    
    display(chart)

    '''nltk_text = nltk.Text(tokens)
    nltk_text.dispersion_plot(top_20['Word'].to_list())'''

dropdown_edizione = widgets.Dropdown(
    options=dropdown_options,
    value=dropdown_options[0],  # Set initial value
    description='Select Edizione:',
    style={'description_width': 'initial'}  # Set description width to full
)  

# Initial display of charts and dropdowns side by side
output1 = widgets.Output()

with output1:
    display(VBox([dropdown_edizione]))
    update_chart1(dropdown_edizione.value)

# Arrange the outputs (charts) horizontally using HBox layout
display(output1)

# Set up event handlers for dropdowns
def on_edizione_change(change):
    with output1:
        clear_output(wait=True)
        display(VBox([dropdown_edizione]))
        update_chart1(change.new)

dropdown_edizione.observe(on_edizione_change, names='value')

Output()

#### TagMe

In [14]:
import requests # For REST calls
import json # for modelling objects in the JSON format

# This line opens the file from the file system, the file is in the same folder of the notebook and it is opened in "read-only mode"
with open("config.json", 'r') as json_file:
    config = json.load(json_file) # load the json object inside the config file
    KEY = config['d4science_KEY'] # this is the key we will be using for REST calls

TAGME_ENDPOINT = "https://tagme.d4science.org/tagme/tag"
LANG = "it" # Also works in italian and german

In [15]:
def query_tagme(text, long_text=False):
    payload = {"text": text, "gcube-token": KEY, "lang": LANG}
    if long_text:
        # long_text is by defaul false, but if specified by the user, we set the window size at 5
        payload["long_text"] = 5
    r = requests.post(TAGME_ENDPOINT, payload)
    if r.status_code != 200:
        raise Exception("Error on text: {}\n{}".format(text, r.text))
    return r.json()
    
# Try changing the min_rho parameter and see how it impacts the returned entities
def get_tagme_entities(tagme_response, min_rho=0.3, min_link=0.3):
    ann = tagme_response["annotations"]
    ann = [a for a in ann if a["rho"] > min_rho] # filter all the annotations with a rho score lower than the threshold
    ann = [a for a in ann if a["link_probability"] > min_link] # filter all the annotations with a rho score lower than the threshold
    return [a["title"] for a in ann if "title" in a] # return just the page titles

In [55]:
text = tutti_i_dati['Testo'].to_list()
print("BEFORE FILTERING")
resp = query_tagme(text, long_text=True) 
before_filtering = [a["title"] for a in resp['annotations'] if "title" in a]
before_filtering

BEFORE FILTERING


['Agricoltura',
 'Allevamento',
 'Ginocchio',
 'Manifestazione',
 'Sciacca',
 'Mattino',
 'Stato federale',
 'Movimento (sociologia)',
 'Agricoltura',
 'Italiani',
 'Stato federale',
 'Comitato (ordinamento civile italiano)',
 'Associazione (diritto)',
 'Terra e Vita',
 'La vita è',
 "Circolo dell'Unione (Firenze)",
 'La Rinascente',
 'Iniziativa (scacchi)',
 'Necessità',
 'Autorità',
 'Stato della materia',
 'La Grave',
 'Sofferenza',
 'Pressione',
 'Costo',
 'Gestione, amministrazione, esercizio',
 'Produzione',
 'Imposta sul reddito delle persone fisiche',
 'Legge di bilancio',
 'Acqua',
 'Lago Arancio',
 'Inquinamento',
 'Rhodophyta',
 'Corteo storico (Palio di Siena)',
 'Trattore agricolo',
 'Provincia di Siena',
 'Unomattina',
 'Sciacca',
 'Compagnia di Gesù',
 "Direttore d'orchestra",
 'Vettore (matematica)',
 'Menfi',
 'Università degli Studi di Milano',
 'Produzione',
 'Valle del Belice',
 'Sinistra Indipendente',
 'Contestazione',
 'Governo',
 'Renato Schifani',
 'Corrente co

In [101]:
# Lista vuota per memorizzare i risultati della trasformazione
risultati = []

# Ciclo for per iterare attraverso ciascuna lista interna
for i in range(0, len(text)):
    resp = query_tagme(text[i], long_text=True) 
    before_filtering = [a["title"] for a in resp['annotations'] if "title" in a]
    risultati.append(before_filtering)

# Visualizzare i risultati
#print(risultati)


[['Agricoltura', 'Allevamento', 'Ginocchio', 'Manifestazione', 'Sciacca', 'Mattino', 'Stato federale', 'Movimento (sociologia)', 'Agricoltura', 'Italiani', 'Stato federale', 'Comitato (ordinamento civile italiano)', 'Associazione (diritto)', 'Terra e Vita', 'La vita è', "Circolo dell'Unione (Firenze)", 'La Rinascente', 'Iniziativa (scacchi)', 'Necessità', 'Autorità', 'Stato della materia', 'La Grave', 'Sofferenza', 'Pressione', 'Costo', 'Gestione, amministrazione, esercizio', 'Produzione', 'Imposta sul reddito delle persone fisiche', 'Legge di bilancio', 'Acqua', 'Lago Arancio', 'Inquinamento', 'Rhodophyta', 'Corteo storico (Palio di Siena)', 'Trattore agricolo', 'Provincia di Siena', 'Unomattina', 'Sciacca', 'Compagnia di Gesù', "Direttore d'orchestra", 'Vettore (matematica)', 'Menfi', 'Università degli Studi di Milano', 'Produzione', 'Valle del Belice', 'Sinistra Indipendente', 'Contestazione', 'Governo', 'Renato Schifani', 'Corrente continua', 'Atteggiamento', 'Tenuta', 'Società (so

In [103]:
# Utilizzare una comprensione di lista per condensare gli elementi
lista_condensata = [elemento for lista_interna in risultati for elemento in lista_interna]

# Visualizzare la lista condensata
#print(lista_condensata)

In [104]:
len(lista_condensata)

55002

In [20]:
print("AFTER FILTERING")
after_filtering = get_tagme_entities(resp)
after_filtering

AFTER FILTERING


['Sciacca',
 'Terra e Vita',
 "Circolo dell'Unione (Firenze)",
 'La Rinascente',
 'Imposta sul reddito delle persone fisiche',
 'Lago Arancio',
 'Rhodophyta',
 'Sciacca',
 'Menfi',
 'Valle del Belice',
 'Partito Democratico (Italia)',
 'Catanzaro',
 'Centro-destra',
 'Sicilia',
 'Palermo',
 'Catanzaro']

In [105]:
import pandas as pd

comuni_df = pd.read_excel('Codici-statistici-e-denominazioni-al-22_01_2024.xlsx')

lista_comuni = comuni_df['Denominazione in italiano'].to_list()

# Convertiamo le liste in set e calcoliamo l'intersezione
intersezione = list(set(lista_condensata) & set(lista_comuni))

# Visualizziamo il risultato
print(intersezione)

['Diano Castello', 'Casalbordino', 'Portico e San Benedetto', 'Zone', 'Camerano', 'Partinico', 'Pisa', 'Drena', 'Orvieto', 'Miggiano', 'Vetralla', 'Erchie', 'Fossacesia', 'Imola', 'Portici', 'Castiglione della Pescaia', 'Montedoro', 'Manfredonia', 'Avellino', 'Pontedera', 'Savignano sul Rubicone', 'Polistena', 'Modigliana', 'Vallo della Lucania', 'Maiolati Spontini', 'Forlì', 'Bastia Umbra', 'Diamante', 'Genova', 'Argenta', 'Fermo', 'Fara Olivana con Sola', 'Orta Nova', 'Trovo', 'Altavilla Silentina', 'Corinaldo', 'Villaricca', 'Quinto di Treviso', 'Capena', 'Terrasini', 'Predappio', 'Trieste', 'Matino', 'San Vito dei Normanni', 'Fagagna', 'Castel San Pietro Terme', 'Scorrano', 'Favara', 'Pizzone', 'Filottrano', 'Contà', 'Castelnuovo del Garda', 'Castel San Pietro Romano', 'Eboli', 'Galluccio', 'Somma Lombardo', 'Resana', 'Viterbo', 'Rimini', 'Cinquefrondi', "Rivolta d'Adda", 'Assisi', 'Piazza Brembana', 'Rosora', 'Nociglia', 'San Giovanni Gemini', 'Terni', 'Caltanissetta', 'Gangi', 'C

In [106]:
len(intersezione)

487

In [120]:
it = pd.read_csv('IT.txt', sep='\t', header=None)
it

,0,1,2,3,4,5,6,7,8,9,10,11
0,IT,67010,Barete,Abruzzi,1,L'Aquila,AQ,NaN,NaN,42.4501,13.2806,4
1,IT,67012,San Giovanni,Abruzzi,1,L'Aquila,AQ,NaN,NaN,42.4642,13.2584,4
2,IT,67012,Cagnano Amiterno,Abruzzi,1,L'Aquila,AQ,NaN,NaN,42.4574,13.2279,4
3,IT,67013,Campotosto,Abruzzi,1,L'Aquila,AQ,NaN,NaN,42.5588,13.3680,4
4,IT,67013,Mascioni,Abruzzi,1,L'Aquila,AQ,NaN,NaN,42.5282,13.3492,4
...,...,...,...,...,...,...,...,...,...,...,...,...
18410,IT,37139,Verona,Veneto,20,Verona,VR,NaN,NaN,45.4299,10.9844,4
18411,IT,37139,Chievo,Veneto,20,Verona,VR,NaN,NaN,45.4631,10.9149,4
18412,IT,37140,Verona,Veneto,20,Verona,VR,NaN,NaN,45.4299,10.9844,4
18413,IT,37141,Verona,Veneto,20,Verona,VR,NaN,NaN,45.4299,10.9844,4


In [121]:
it = it.rename(columns={1: 'droppare', 2: 'Comune', 3: 'Regione', 4: 'codice_regione', 9: 'Lat', 10: 'Long'})
it = it.drop(columns=['droppare'])
it_filtrato = it[['Comune', 'Regione', 'codice_regione', 'Lat', 'Long']]
it_filtrato

,Comune,Regione,codice_regione,Lat,Long
0,Barete,Abruzzi,1,42.4501,13.2806
1,San Giovanni,Abruzzi,1,42.4642,13.2584
2,Cagnano Amiterno,Abruzzi,1,42.4574,13.2279
3,Campotosto,Abruzzi,1,42.5588,13.3680
4,Mascioni,Abruzzi,1,42.5282,13.3492
...,...,...,...,...,...
18410,Verona,Veneto,20,45.4299,10.9844
18411,Chievo,Veneto,20,45.4631,10.9149
18412,Verona,Veneto,20,45.4299,10.9844
18413,Verona,Veneto,20,45.4299,10.9844


In [122]:
it_distinct = it_filtrato.drop_duplicates()
it_distinct

,Comune,Regione,codice_regione,Lat,Long
0,Barete,Abruzzi,1,42.4501,13.2806
1,San Giovanni,Abruzzi,1,42.4642,13.2584
2,Cagnano Amiterno,Abruzzi,1,42.4574,13.2279
3,Campotosto,Abruzzi,1,42.5588,13.3680
4,Mascioni,Abruzzi,1,42.5282,13.3492
...,...,...,...,...,...
18392,Quinzano,Veneto,20,45.4299,10.9844
18396,Avesa,Veneto,20,45.4667,11.0000
18401,San Michele Extra,Veneto,20,45.4311,11.0512
18409,San Massimo All'Adige,Veneto,20,45.4465,10.9497


In [123]:
intersezione_df = pd.DataFrame(intersezione, columns=['Comune'])
intersezione_df

,Comune
0,Diano Castello
1,Casalbordino
2,Portico e San Benedetto
3,Zone
4,Camerano
...,...
482,Bolzano
483,San Marco Evangelista
484,Arezzo
485,Narni


In [124]:
df_inner = pd.merge(it_distinct, intersezione_df, on='Comune', how='inner')
df_inner

,Comune,Regione,codice_regione,Lat,Long
0,Pratola Peligna,Abruzzi,1,42.0970,13.8747
1,Celano,Abruzzi,1,42.0808,13.5170
2,Cerchio,Abruzzi,1,42.0636,13.6001
3,L'Aquila,Abruzzi,1,42.3506,13.3995
4,Villamagna,Abruzzi,1,42.3298,14.2369
...,...,...,...,...,...
434,Venezia,Veneto,20,45.4371,12.3327
435,Lonigo,Veneto,20,45.3858,11.3840
436,Vicenza,Veneto,20,45.5467,11.5475
437,Rivoli Veronese,Veneto,20,45.5725,10.8118


In [186]:
import altair as alt

urlGeo='https://raw.githubusercontent.com/openpolis/geojson-italy/master/topojson/limits_IT_all.topo.json'
italyGeoRegions=alt.topo_feature(urlGeo,'regions')

italyBaseMap=alt.Chart(italyGeoRegions).mark_geoshape(
    fill='lightgray', stroke='white', strokeWidth=1
)

italyBaseMap

alt.Chart(...)

In [188]:
bubbles=alt.Chart(df_inner).mark_circle().encode(
        latitude='Lat:Q',
        longitude='Long:Q',
        tooltip='Comune:N'
        #size='superficie totale - ettari:Q' # come fare i pallini più grandi (hint: cerco range nella documentazione)

    )

(italyBaseMap + bubbles).properties(width=600,height=600)

alt.LayerChart(...)

In [183]:
# Conta le occorrenze distinte nella colonna 'A' e trasforma il risultato in un DataFrame
df_count = df_inner['Regione'].value_counts().reset_index()

# Rinomina le colonne del nuovo DataFrame
df_count.columns = ['denominazione_regione', 'Conteggio']
df_count['denominazione_regione'] = df_count['denominazione_regione'].replace(['Abruzzi'],['Abruzzo'])
df_count

,denominazione_regione,Conteggio
0,Emilia-Romagna,62
1,Campania,49
2,Lombardia,49
3,Puglia,48
4,Sicilia,38
5,Lazio,35
6,Veneto,25
7,Marche,24
8,Abruzzo,21
9,Toscana,18


In [150]:
import pandas as pd
import requests

# Lista delle regioni italiane
regioni_italiane = [
    'Abruzzo', 'Basilicata', 'Calabria', 'Campania', 'Emilia-Romagna',
    'Friuli-Venezia Giulia', 'Lazio', 'Liguria', 'Lombardia', 'Marche',
    'Molise', 'Piemonte', 'Puglia', 'Sardegna', 'Sicilia', 'Toscana',
    'Trentino-Alto Adige', 'Umbria', 'Valle d\'Aosta', 'Veneto'
]

# Funzione per ottenere le coordinate geografiche utilizzando un servizio di geocodifica (es. Nominatim di OpenStreetMap)
def ottieni_coordinate(regione):
    url = f"https://nominatim.openstreetmap.org/search.php?q={regione}, Italy&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data:
            lat = data[0]['lat']
            lon = data[0]['lon']
            return {'Regione': regione, 'lat': lat, 'long': lon}
    return None

# Creazione del DataFrame delle coordinate geografiche delle regioni italiane
coordinate_regioni = [ottieni_coordinate(regione) for regione in regioni_italiane]
coordinate_df = pd.DataFrame(coordinate_regioni)

# Visualizza il DataFrame
coordinate_df

,Regione,lat,long
0,Abruzzo,42.227681,13.854983
1,Basilicata,40.500571,16.081953
2,Calabria,39.0565974,16.5249864
3,Campania,40.860672,14.843984
4,Emilia-Romagna,44.525696,11.039437
5,Friuli-Venezia Giulia,46.151042,13.055904
6,Lazio,41.9808038,12.7662312
7,Liguria,44.4777617,8.7026296
8,Lombardia,45.5703694,9.7732524
9,Marche,43.3458388,13.1415872


In [164]:
import pandas as pd
import altair as alt

# Creazione di un DataFrame di esempio con i codici ISTAT delle regioni italiane
dati = {
    'Regione': ['Abruzzo', 'Basilicata', 'Calabria', 'Campania', 'Emilia-Romagna', 'Friuli-Venezia Giulia',
                'Lazio', 'Liguria', 'Lombardia', 'Marche', 'Molise', 'Piemonte', 'Puglia', 'Sardegna',
                'Sicilia', 'Toscana', 'Trentino-Alto Adige', 'Umbria', "Valle d'Aosta", 'Veneto'],
    'codice_regione': [13, 17, 18, 15, 8, 6, 12, 7, 3, 11, 14, 1, 16, 20, 19, 9, 4, 10, 2, 5]
}

df_codice_regione = pd.DataFrame(dati)
df_codice_regione

,Regione,codice_regione
0,Abruzzo,13
1,Basilicata,17
2,Calabria,18
3,Campania,15
4,Emilia-Romagna,8
5,Friuli-Venezia Giulia,6
6,Lazio,12
7,Liguria,7
8,Lombardia,3
9,Marche,11


In [167]:
df_final = df_count.merge(coordinate_df, on='Regione', how='inner')
df_final_2 = df_final.merge(df_codice_regione, on='Regione', how='inner')
df_final_2

,Regione,Conteggio,lat,long,codice_regione
0,Emilia-Romagna,62,44.525696,11.039437,8
1,Campania,49,40.860672,14.843984,15
2,Lombardia,49,45.5703694,9.7732524,3
3,Puglia,48,40.9842539,16.6210027,16
4,Sicilia,38,37.587794,14.155048,19
5,Lazio,35,41.9808038,12.7662312,12
6,Veneto,25,45.6476663,11.8665254,5
7,Marche,24,43.3458388,13.1415872,11
8,Abruzzo,21,42.227681,13.854983,13
9,Toscana,18,43.4586541,11.1389204,9


In [216]:
apiCovid='https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'
df=pd.read_csv(apiCovid, parse_dates=['data'])[['denominazione_regione', 'codice_regione', 'lat', 'long']].drop_duplicates()
df

,denominazione_regione,codice_regione,lat,long
0,Abruzzo,13,42.351222,13.398438
1,Basilicata,17,40.639471,15.805148
2,Calabria,18,38.905976,16.594402
3,Campania,15,40.839566,14.250850
4,Emilia-Romagna,8,44.494367,11.341721
5,Friuli Venezia Giulia,6,45.649435,13.768136
6,Lazio,12,41.892770,12.483667
7,Liguria,7,44.411493,8.932699
8,Lombardia,3,45.466794,9.190347
9,Marche,11,43.616760,13.518875


In [217]:
df_final = df.merge(df_count, on='denominazione_regione', how='inner')
df_final

,denominazione_regione,codice_regione,lat,long,Conteggio
0,Abruzzo,13,42.351222,13.398438,21
1,Basilicata,17,40.639471,15.805148,3
2,Calabria,18,38.905976,16.594402,15
3,Campania,15,40.839566,14.250850,49
4,Emilia-Romagna,8,44.494367,11.341721,62
5,Lazio,12,41.892770,12.483667,35
6,Liguria,7,44.411493,8.932699,10
7,Lombardia,3,45.466794,9.190347,49
8,Marche,11,43.616760,13.518875,24
9,Molise,14,41.557748,14.659161,3


In [222]:
alt.Chart(italyGeoRegions).mark_geoshape(
    stroke='#fff', strokeWidth=0.25
).transform_lookup(
    lookup='properties.reg_istat_code_num', from_=alt.LookupData(data=df_final_2, key='codice_regione', fields=['Regione', 'Conteggio'])
).encode(
    color= alt.Color('Conteggio:Q', scale=alt.Scale(scheme='oranges', type='quantize')),
    tooltip= [
              alt.Tooltip('Conteggio:Q'),
              alt.Tooltip('Regione:N')
              #alt.Tooltip('data:N')
    ]
).project(
    type='mercator'
).properties(
    width=400,
    height=400
).configure_view(
    stroke=None
)

alt.Chart(...)